In [8]:
import os
import openai
from dotenv import load_dotenv
load_dotenv()
# Config for Azure OpenAI.
OPENAI_API_TYPE = "azure"
OPENAI_API_BASE = os.getenv("OPENAI_API_BASE")
OPENAI_API_VERSION = "2023-07-01-preview"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [1]:
import sqlite3

# Connect to a new database file called "mydatabase.db"
conn = sqlite3.connect('Chinook.db')

# Close the database connection
conn.close()

In [6]:
import sqlite3

# Connect to a new database file
conn = sqlite3.connect('Chinook.db')

# Open the SQL script file and read its contents
with open('../data/Chinook_Sqlite.sql', 'r', encoding='utf-8') as f:  
    sql_script = f.read()  

# Execute the SQL script in the new database
conn.executescript(sql_script)

# Close the database connection
conn.close()

In [7]:
import sqlite3

# Connect to an existing database file
conn = sqlite3.connect('Chinook.db')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Execute a SELECT query
cursor.execute('SELECT * FROM Artist LIMIT 10')

# Fetch the query results
rows = cursor.fetchall()

# Print the results
for row in rows:
    print(row)

# Close the cursor and database connection
cursor.close()
conn.close()

(1, 'AC/DC')
(2, 'Accept')
(3, 'Aerosmith')
(4, 'Alanis Morissette')
(5, 'Alice In Chains')
(6, 'Antônio Carlos Jobim')
(7, 'Apocalyptica')
(8, 'Audioslave')
(9, 'BackBeat')
(10, 'Billy Cobham')


In [17]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import AgentExecutor

In [18]:
llm4 = AzureChatOpenAI(
    openai_api_base= OPENAI_API_BASE,
    openai_api_version= OPENAI_API_VERSION,
    deployment_name="gpt-4-32k",
    temperature=0,
    openai_api_key= OPENAI_API_KEY ,
    openai_api_type = OPENAI_API_TYPE,
)

In [21]:
model = AzureOpenAI(
    openai_api_base= OPENAI_API_BASE,
    openai_api_version= OPENAI_API_VERSION,
    deployment_name="gpt-35-turbo-instruct",
    temperature=0,
    openai_api_key= OPENAI_API_KEY,
    openai_api_type = OPENAI_API_TYPE,
)
model("tell me a joke")

"\n\nWhy couldn't the bicycle stand up by itself?\n\nBecause it was two-tired!"

In [29]:
db = SQLDatabase.from_uri("sqlite:///./Chinook.db")
toolkit = SQLDatabaseToolkit(db=db, llm= llm4)

agent_executor = create_sql_agent(
    llm =llm4,
    toolkit=toolkit,
    verbose=True
)

In [30]:
agent_executor.run("Describe the playlisttrack table")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought:The PlaylistTrack table is in the database. I should now query the schema of the PlaylistTrack table.
Action: sql_db_schema
Action Input: "PlaylistTrack"
Observation: 
CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/
Thought:I now know the final answer
Final Answer: The "PlaylistTrack" table has two columns: "PlaylistId" and "TrackId". Both are integers and are not null. They are also the primary keys of the table. The "TrackId" column references the "TrackId" in the "Track" table, and the "Play

'The "PlaylistTrack" table has two columns: "PlaylistId" and "TrackId". Both are integers and are not null. They are also the primary keys of the table. The "TrackId" column references the "TrackId" in the "Track" table, and the "PlaylistId" column references the "PlaylistId" in the "Playlist" table. Here are some sample data: \n1. PlaylistId: 1, TrackId: 3402\n2. PlaylistId: 1, TrackId: 3389\n3. PlaylistId: 1, TrackId: 3390'

### Example: describing a table, recovering from an error

In [31]:
agent_executor.run("Describe the playlistsong table")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ''
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought:The 'PlaylistTrack' table seems to be the most relevant to the question. I should query the schema of this table.
Action: sql_db_schema
Action Input: 'PlaylistTrack'
Observation: Error: table_names {"'PlaylistTrack'"} not found in database
Thought:It seems I made a mistake with the table name. It should be 'PlaylistTrack' without the quotes. I'll try again.
Action: sql_db_schema
Action Input: PlaylistTrack
Observation: 
CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/
Thought:I now know the fina

"The 'PlaylistTrack' table has two columns: 'PlaylistId' and 'TrackId'. Both are integers and are not null. They are also the primary keys of the table. The 'TrackId' column references the 'TrackId' in the 'Track' table and the 'PlaylistId' column references the 'PlaylistId' in the 'Playlist' table. Here are some sample rows from the 'PlaylistTrack' table: \n1. PlaylistId: 1, TrackId: 3402\n2. PlaylistId: 1, TrackId: 3389\n3. PlaylistId: 1, TrackId: 3390"

### Example: running queries

In [32]:
agent_executor.run("List the total sales per country. Which country's customers spent the most?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought:The relevant tables for this query seem to be the Customer and Invoice tables. I should check their schemas to see what columns they have.
Action: sql_db_schema
Action Input: "Customer, Invoice"
Observation: 
CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	Co

'The country whose customers spent the most is USA with a total sales of 523.06.'

In [33]:
agent_executor.run("Show the total number of tracks in each playlist. The Playlist name should be included in the result.")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought:The relevant tables for this query seem to be Playlist and PlaylistTrack. I should check the schema of these tables to understand their structure and relationships.
Action: sql_db_schema
Action Input: "Playlist, PlaylistTrack" 
Observation: 
CREATE TABLE "Playlist" (
	"PlaylistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("PlaylistId")
)

/*
3 rows from Playlist table:
PlaylistId	Name
1	Music
2	Movies
3	TV Shows
*/


CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/
Thought:The Pla

"The total number of tracks in each playlist are as follows: '90’s Music' has 1477 tracks, 'Brazilian Music' has 39 tracks, 'Classical' has 75 tracks, 'Classical 101 - Deep Cuts' has 25 tracks, 'Classical 101 - Next Steps' has 25 tracks, 'Classical 101 - The Basics' has 25 tracks, 'Grunge' has 15 tracks, 'Heavy Metal Classic' has 26 tracks, 'Music' has 6580 tracks, 'Music Videos' has 1 track."